In [5]:
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge
import pandas as pd

In [6]:
df = pd.read_csv("obat_alodokter_new.csv")
df.head()
df_obat_for_eval = df.drop(['nama_obat'],axis=1)

In [7]:
import re

def clean_text(text):
    text = re.sub(r'\n+', ' ', text)  # hapus banyak \n
    text = re.sub(r'\s+', ' ', text)   # hapus spasi berlebihan
    return text.strip()

df['cleaned_reference'] = df['content'].apply(clean_text)

# Misal kamu punya generated output juga (contoh dummy di sini)
df['generated_text'] = df['cleaned_reference'].apply(lambda x: x.lower())

In [10]:
df_obat_for_bleu = df.drop(['nama_obat','generated_text','content'],axis=1)
df_obat_for_ndcg = df.drop(['generated_text','content'],axis=1)

In [11]:
df_obat_for_bleu

,cleaned_reference
0,Zyloric Zyloric adalah obat untuk menurunkan k...
1,Zovirax Krim Zovirax krim adalah obat oles yan...
2,Zovigen Zovigen adalah obat oles yang bermanfa...
3,Zoralin Krim Zoralin krim adalah obat oles unt...
4,Zoralin Zoralin adalah tablet antijamur berbah...
...,...
2168,Acarbose Acarbose adalah obat untuk menurunkan...
2169,Acalabrutinib Acalabrutinib adalah obat kemote...
2170,Abrocitinib Abrocitinib adalah obat untuk meng...
2171,Abemaciclib Abemaciclib adalah obat kemoterapi...


In [ ]:
df_obat_for_bleu['generated_text'] = None

# Isi cuma di row index 0
df_obat_for_bleu.loc[0, 'generated_text'] = '1. Rekomendasi obat:  - Sumatriptan Dosis, 50 mg, dapat diulang setelah 2 jam jika gejala tidak membaik; maksimum 200 mg per hari. Aturan pakai, Dikonsumsi saat gejala migrain muncul. Efek samping,  Mual, peningkatan tekanan darah, rasa terbakar atau kesemutan di area wajah.  - Ibuprofen Dosis: 400 mg setiap 4-6 jam sesuai kebutuhan; maksimum 1200 mg per hari.  Aturan pakai: Diminum dengan makanan untuk mengurangi iritasi lambung.  Efek samping: Mual, sakit perut, pusing, atau alergi.  Gejala terkait lainnya: Sakit kepala, muntah, sensitivitas terhadap cahaya.'  
df_obat_for_bleu.loc[1, 'generated_text'] = '2. Rekomendasi obat:  - Rehidrasi Oral  Dosis: Sesuai petunjuk pada kemasan. Aturan pakai: Diberikan secara bertahap untuk menggantikan cairan yang hilang. Efek samping: Jarang terjadi jika digunakan sesuai petunjuk. Gejala terkait lainnya: Diare, mual, muntah.'
df_obat_for_bleu.loc[2, 'generated_text'] = '3. Rekomendasi obat:  - Paracetamol  Dosis: 500 mg setiap 4-6 jam sesuai kebutuhan; maksimum 3000 mg per hari.  Aturan pakai: Diminum setiap kali merasakan demam atau nyeri.  Efek samping: Jarang terjadi jika digunakan sesuai petunjuk.  Gejala terkait lainnya: Pilek, sakit tenggorokan, demam.'
df_obat_for_bleu.loc[3, 'generated_text'] = '4. Rekomendasi obat:  - Meclizine  Dosis: 25 mg hingga 100 mg per hari, sesuai kebutuhan.  Aturan pakai: Diminum sebelum aktivitas yang dapat memicu gejala.  Efek samping: Mengantuk, mulut kering, kelelahan. Gejala terkait lainnya: Kehilangan keseimbangan, muntah'

In [27]:
import pandas as pd
import json

# Fungsi untuk mengubah hasil ke dalam format yang sesuai untuk DataFrame
def format_result_to_text(user_data):
    formatted_text = ""
    for idx, record in enumerate(user_data['agent']):
        nomor = idx + 1  # Menentukan nomor urut obat
        formatted_text += f"{nomor}. Rekomendasi obat:\n"
        formatted_text += f" - {record.get('-_nama_obat', 'Nama Obat Tidak Tersedia')} Dosis: {record.get('-_dosis', 'Dosis Tidak Tersedia')}\n"
        formatted_text += f"   Aturan pakai: {record.get('-_aturan_pakai', 'Aturan Pakai Tidak Tersedia')}\n"
        formatted_text += f"   Efek samping: {record.get('-_efek_samping_obat', 'Efek Samping Tidak Tersedia')}\n\n"
    return formatted_text.strip()

# Membaca data dari memory.json
with open('memory.json', 'r') as file:
    data = json.load(file)

# Membuat DataFrame kosong untuk memasukkan hasil
df_obat_for_bleu = pd.DataFrame({
    'generated_text': [None] * 4  # Isi data hanya di index yang diperlukan
})

# Mengambil data dari pengguna pertama (5708441153) atau bisa disesuaikan sesuai kebutuhan
user_id = "7116300622"
if user_id in data:
    user_data = data[user_id][0]  # Mengambil data pengguna pertama
    formatted_text = format_result_to_text(user_data)
    
    # Masukkan ke dalam DataFrame
    df_obat_for_bleu.loc[0, 'generated_text'] = formatted_text

# Cetak hasil DataFrame
print(df_obat_for_bleu)

                                      generated_text
0  1. Rekomendasi obat:\n - Nama Obat Tidak Terse...
1                                               None
2                                               None
3                                               None


In [14]:
bleu_scores = []
rouge_scores = []

rouge = Rouge()

In [15]:
for idx, row in df.iterrows():
    reference = [row['cleaned_reference'].split()]  # list of list
    candidate = row['generated_text'].split()

    # Hitung BLEU
    bleu = sentence_bleu(reference, candidate)
    bleu_scores.append(bleu)

    # Hitung ROUGE
    rouge_score = rouge.get_scores(row['generated_text'], row['cleaned_reference'])[0]
    rouge_scores.append(rouge_score)

# Rata-rata skor
avg_bleu = sum(bleu_scores) / len(bleu_scores)
avg_rouge_1 = sum([r['rouge-1']['f'] for r in rouge_scores]) / len(rouge_scores)
avg_rouge_2 = sum([r['rouge-2']['f'] for r in rouge_scores]) / len(rouge_scores)
avg_rouge_l = sum([r['rouge-l']['f'] for r in rouge_scores]) / len(rouge_scores)

print("Average BLEU:", avg_bleu)
print("Average ROUGE-1:", avg_rouge_1)
print("Average ROUGE-2:", avg_rouge_2)
print("Average ROUGE-L:", avg_rouge_l)

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import json
import math

list_obat = df_obat_for_ndcg['nama_obat'].tolist()

with open("memory.json", "r", encoding="utf-8") as f:
    data = json.load(f)

def map_penyakit_ke_obat(df_obat, penyakit):
    matched_obat = []
    for _, row in df_obat.iterrows():
        if isinstance(row['cleaned_reference'], str) and penyakit.lower() in row['cleaned_reference'].lower():
            matched_obat.append(row['nama_obat'])
    return matched_obat

def compute_ndcg_at_k(true_relevant, recommended_items, k):
    def dcg(relevances):
        return sum((1 / math.log2(i + 2)) for i, rel in enumerate(relevances) if rel > 0)
    
    relevance_scores = [1 if item in true_relevant else 0 for item in recommended_items[:k]]
    ideal_relevance = sorted(relevance_scores, reverse=True)

    dcg_score = dcg(relevance_scores)
    idcg_score = dcg(ideal_relevance)
    return dcg_score / idcg_score if idcg_score != 0 else 0.0

for user_id, entries in data.items():
    if user_id in ["started_users", "history_backup"]:
        continue

    for entry in entries:
        user_input = entry.get("user", "")
        agent_outputs = entry.get("agent", [])

        # List untuk penyakit yang terdeteksi dan rekomendasi obat
        penyakit_list = []
        recommended_items = []

        for agent in agent_outputs:
            nama_penyakit = agent.get("nama_penyakit", "")
            rekom_obat = agent.get("rekomendasi_obat", "")

            if nama_penyakit:
                penyakit_list.append(nama_penyakit)

            if rekom_obat and "belum ada" not in rekom_obat.lower():
                recommended_items.append(rekom_obat)

        # TRUE RELEVANT: semua obat yang cocok dengan penyakit
        true_relevant = []
        for penyakit in penyakit_list:
            true_relevant.extend(map_penyakit_ke_obat(df_obat_for_ndcg, penyakit))

        # Hapus duplikat kalau ada
        true_relevant = list(set(true_relevant))

        # Hitung skor NDCG@5
        ndcg_score = compute_ndcg_at_k(true_relevant, recommended_items, k=5)

        # Tampilkan hasil untuk setiap user dan setiap penyakit
        print(f"User ID: {user_id}")
        print(f"User Input: {user_input}")
        print(f"Penyakit Terdeteksi: {penyakit_list}")
        print(f"Recommended Obat: {recommended_items}")
        print(f"True Relevant Obat: {true_relevant}")
        print(f"NDCG@5: {ndcg_score:.4f}\n")

User ID: 5708441153
User Input: saya sulit tidur dan kadang mengalami sesak napas
Penyakit Terdeteksi: ['Sleep Apnea', 'Asthma', 'Anxiety Disorders']
Recommended Obat: ['Modafinil']
True Relevant Obat: ['Zolmia', 'Somatropin', 'Zolastin', 'Alprazolam', 'Mesterolone', 'Paliperidone', 'Muscle Relaxant', 'Donepezil', 'Codikaf', 'Esketamine Semprot Hidung', 'Aripiprazole', 'Tapentadol', 'Levodopa', 'Risperidone', 'Fingolimod', 'Butorphanol', 'Morfin', 'Zolpidem', 'Primidone', 'Midazolam', 'Clonazepam', 'Olanzapine', 'Dekongestan', 'Testosterone', 'Buprenorphine', 'Sanmag', 'Codeine', 'Tramadol-Paracetamol', 'Neripros', 'Xanax', 'Sufentanil', 'Metoprolol', 'Temazepam', 'Acebutolol', 'Remifentanil', 'Hydromorphone', 'Hydrocodone', 'Promethazine', 'Secobarbital', 'Nimetazepam', 'Nalbuphine', 'Diazepam', 'Lorazepam', 'Tramadol', 'Brexpiprazole', 'Benzodiazepine', 'Quetiapine', 'Modafinil', 'Asenapine', 'Pramipexole', 'Oxycodone', 'Dumolid', 'Bromazepam', 'Bronkodilator', 'Lexzepam', 'Diazepam 